# Seldon V2 Non Kubernetes Pipeline Examples


 * Build if needed and place `seldon` binary in your path
    * run `make build-seldon` from operator folder and add bin folder to PATH
 * Run Seldon V2 `make deploy-local` from top level folder

In [1]:
!which seldon

/home/clive/work/scv2/seldon-core-v2/operator/bin/seldon


## Model Chaining

In [2]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow


In [3]:
!seldon model load -f ./models/tfsimple1.yaml 
!seldon model load -f ./models/tfsimple2.yaml 

{}
{}


In [4]:
!seldon model status --model-name tfsimple1 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple2 -w ModelAvailable | jq .

{
  "modelName": "tfsimple1",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:46:23.537718480Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:46:23.537718480Z"
      },
      "modelDefn": {
        "meta": {
          "name": "tfsimple1",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/simple",
          "requirements": [
            "tensorflow"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "tfsimple2",
  "versions": [
    {
      "version": 1,
      "serverN

In [5]:
!cat ./pipelines/tfsimples.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimples
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
      inputs:
      - tfsimple1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple1.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple2


In [6]:
!seldon pipeline load -f ./pipelines/tfsimples.yaml

{}


In [7]:
!seldon pipeline status -p tfsimples -w PipelineReady| jq .

{
  "pipelineName": "tfsimples",
  "versions": [
    {
      "pipeline": {
        "name": "tfsimples",
        "steps": [
          {
            "name": "tfsimple1"
          },
          {
            "name": "tfsimple2",
            "inputs": [
              "tfsimple1.outputs"
            ],
            "tensorMap": {
              "tfsimple1.outputs.OUTPUT0": "INPUT0",
              "tfsimple1.outputs.OUTPUT1": "INPUT1"
            }
          }
        ],
        "output": {
          "steps": [
            "tfsimple2.outputs"
          ]
        },
        "kubernetesMeta": {
          "namespace": "seldon-mesh"
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-04-26T08:46:57.656002122Z"
      }
    }
  ]
}


In [8]:
!seldon pipeline infer -p tfsimples \
    '{"inputs":[{"name":"INPUT0","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}' | jq .

{
  "model_name": "",
  "outputs": [
    {
      "data": null,
      "name": "OUTPUT0",
      "shape": [
        1,
        16
      ],
      "datatype": "INT32"
    },
    {
      "data": null,
      "name": "OUTPUT1",
      "shape": [
        1,
        16
      ],
      "datatype": "INT32"
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA=="
  ]
}


In [9]:
!seldon pipeline infer -p tfsimples --inference-mode grpc \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABY

In [10]:
!seldon pipeline unload -p tfsimples

{}


In [11]:
!seldon model unload --model-name tfsimple1
!seldon model unload --model-name tfsimple2

{}
{}


## Model Join

In [12]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple3
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow


In [13]:
!seldon model load -f ./models/tfsimple1.yaml 
!seldon model load -f ./models/tfsimple2.yaml 
!seldon model load -f ./models/tfsimple3.yaml 

{}
{}
{}


In [14]:
!seldon model status --model-name tfsimple1 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple2 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple3 -w ModelAvailable | jq .

{
  "modelName": "tfsimple1",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:47:26.494589647Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:47:26.494589647Z"
      },
      "modelDefn": {
        "meta": {
          "name": "tfsimple1",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/simple",
          "requirements": [
            "tensorflow"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "tfsimple2",
  "versions": [
    {
      "version": 1,
      "serverN

In [15]:
!cat ./pipelines/tfsimples-join.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: join
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
    - name: tfsimple3      
      inputs:
      - tfsimple1.outputs.OUTPUT0
      - tfsimple2.outputs.OUTPUT1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple2.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple3


In [16]:
!seldon pipeline load -f ./pipelines/tfsimples-join.yaml

{}


In [17]:
!seldon pipeline status -p join -w PipelineReady | jq .

{
  "pipelineName": "join",
  "versions": [
    {
      "pipeline": {
        "name": "join",
        "steps": [
          {
            "name": "tfsimple1"
          },
          {
            "name": "tfsimple2"
          },
          {
            "name": "tfsimple3",
            "inputs": [
              "tfsimple1.outputs.OUTPUT0",
              "tfsimple2.outputs.OUTPUT1"
            ],
            "tensorMap": {
              "tfsimple1.outputs.OUTPUT0": "INPUT0",
              "tfsimple2.outputs.OUTPUT1": "INPUT1"
            }
          }
        ],
        "output": {
          "steps": [
            "tfsimple3.outputs"
          ]
        },
        "kubernetesMeta": {
          "namespace": "seldon-mesh"
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-04-26T08:47:58.456508443Z"
      }
    }
  ]
}


In [19]:
!seldon pipeline infer -p join --inference-mode grpc \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABY

In [20]:
!seldon pipeline unload -p join

{}


In [21]:
!seldon model unload --model-name tfsimple1
!seldon model unload --model-name tfsimple2
!seldon model unload --model-name tfsimple3

{}
{}
{}


## Conditional

In [22]:
!cat ./models/conditional.yaml
!cat ./models/add10.yaml
!cat ./models/mul10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: conditional
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/conditional"
  requirements:
  - triton-python
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: add10
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/add10"
  requirements:
  - triton-python
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: mul10
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/mul10"
  requirements:
  - triton-python


In [23]:
!seldon model load -f ./models/conditional.yaml 
!seldon model load -f ./models/add10.yaml 
!seldon model load -f ./models/mul10.yaml 

{}
{}
{}


In [24]:
!seldon model status --model-name conditional -w ModelAvailable | jq .
!seldon model status --model-name add10 -w ModelAvailable | jq .
!seldon model status --model-name mul10 -w ModelAvailable | jq .

{
  "modelName": "conditional",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:49:03.427151223Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:49:03.427151223Z"
      },
      "modelDefn": {
        "meta": {
          "name": "conditional",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/conditional",
          "requirements": [
            "triton-python"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "add10",
  "versions": [
    {
      "version": 1,
      

In [25]:
!cat ./pipelines/conditional.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimple-conditional
  namespace: seldon-mesh
spec:
  steps:
  - name: conditional
  - name: mul10
    inputs:
    - conditional.outputs.OUTPUT0
    tensorMap:
      conditional.outputs.OUTPUT0: INPUT
  - name: add10
    inputs:
    - conditional.outputs.OUTPUT1
    tensorMap:
      conditional.outputs.OUTPUT1: INPUT
  output:
    steps:
    - mul10
    - add10
    stepsJoin: any


In [26]:
!seldon pipeline load -f ./pipelines/conditional.yaml

{}


In [27]:
!seldon pipeline status -p tfsimple-conditional -w PipelineReady | jq .

{
  "pipelineName": "tfsimple-conditional",
  "versions": [
    {
      "pipeline": {
        "name": "tfsimple-conditional",
        "steps": [
          {
            "name": "add10",
            "inputs": [
              "conditional.outputs.OUTPUT1"
            ],
            "tensorMap": {
              "conditional.outputs.OUTPUT1": "INPUT"
            }
          },
          {
            "name": "conditional"
          },
          {
            "name": "mul10",
            "inputs": [
              "conditional.outputs.OUTPUT0"
            ],
            "tensorMap": {
              "conditional.outputs.OUTPUT0": "INPUT"
            }
          }
        ],
        "output": {
          "steps": [
            "mul10.outputs",
            "add10.outputs"
          ],
          "stepsJoin": "ANY"
        },
        "kubernetesMeta": {
          "namespace": "seldon-mesh"
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
       

In [30]:
!seldon pipeline infer -p tfsimple-conditional --inference-mode grpc \
 '{"model_name":"outlier","inputs":[{"name":"CHOICE","contents":{"int_contents":[0]},"datatype":"INT32","shape":[1]},{"name":"INPUT0","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]},{"name":"INPUT1","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT",
      "datatype": "FP32",
      "shape": [
        "4"
      ],
      "contents": {
        "fp32Contents": [
          10,
          20,
          30,
          40
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AAAgQQAAoEEAAPBBAAAgQg=="
  ]
}


In [31]:
!seldon pipeline infer -p tfsimple-conditional --inference-mode grpc \
 '{"model_name":"outlier","inputs":[{"name":"CHOICE","contents":{"int_contents":[1]},"datatype":"INT32","shape":[1]},{"name":"INPUT0","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]},{"name":"INPUT1","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT",
      "datatype": "FP32",
      "shape": [
        "4"
      ],
      "contents": {
        "fp32Contents": [
          11,
          12,
          13,
          14
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AAAwQQAAQEEAAFBBAABgQQ=="
  ]
}


In [32]:
!seldon pipeline unload -p tfsimple-conditional

{}


In [33]:
!seldon model unload --model-name conditional
!seldon model unload --model-name add10
!seldon model unload --model-name mul10

{}
{}
{}


## Error
An example which errors is arguments sum to greater than 100

In [34]:
!cat ./models/outlier-error.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: outlier-error
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/outlier"
  requirements:
  - triton-python


In [35]:
!seldon model load -f ./models/outlier-error.yaml 

{}


In [36]:
!seldon model status --model-name outlier-error -w ModelAvailable | jq .

{
  "modelName": "outlier-error",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:50:11.857321912Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:50:11.857321912Z"
      },
      "modelDefn": {
        "meta": {
          "name": "outlier-error",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/outlier",
          "requirements": [
            "triton-python"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}


In [37]:
!cat ./pipelines/error.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: error
spec:
  steps:
    - name: outlier-error
  output:
    steps:
    - outlier-error


In [38]:
!seldon pipeline load -f ./pipelines/error.yaml

{}


In [39]:
!seldon pipeline status -p error -w PipelineReady | jq .

{
  "pipelineName": "error",
  "versions": [
    {
      "pipeline": {
        "name": "error",
        "steps": [
          {
            "name": "outlier-error"
          }
        ],
        "output": {
          "steps": [
            "outlier-error.outputs"
          ]
        },
        "kubernetesMeta": {}
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-04-26T08:50:35.200724370Z"
      }
    }
  ]
}


In [40]:
!seldon pipeline infer -p error --inference-mode grpc \
    '{"model_name":"outlier","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT",
      "datatype": "FP32",
      "shape": [
        "4"
      ],
      "contents": {
        "fp32Contents": [
          10,
          20,
          30,
          40
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AAAgQQAAoEEAAPBBAAAgQg=="
  ]
}


In [41]:
!seldon pipeline infer -p error --inference-mode grpc \
    '{"model_name":"outlier","inputs":[{"name":"INPUT","contents":{"fp32_contents":[100,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

Error: rpc error: code = FailedPrecondition desc = rpc error: code = Internal desc = Failed to process the request(s) for model instance 'outlier-error_1_0', message: TritonModelException: Outlier. Input sums to greater than 100

At:
  /mnt/agent/models/outlier-error_1/1/model.py(108): execute

Usage:
  seldon pipeline infer [flags]

Flags:
  -f, --file-path string        inference payload file
  -h, --help                    help for infer
      --inference-host string   seldon inference host (default "0.0.0.0")
      --inference-mode string   inference mode rest or grpc (default "rest")
      --inference-port int      seldon scheduler port (default 9000)
  -i, --iterations int          inference iterations (default 1)
  -p, --pipeline-name string    pipeline name for inference

Global Flags:
  -r, --show-request    show request
  -o, --show-response   show response (default true)

rpc error: code = FailedPrecondition desc = rpc error: code = Internal desc = Failed to process the requ

In [42]:
!seldon pipeline unload -p error

{}


In [43]:
!seldon model unload --model-name outlier-error

{}


## Outlier
An example runs only if no outlier

In [44]:
!cat ./models/outlier-error.yaml
!cat ./models/add10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: outlier-error
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/outlier"
  requirements:
  - triton-python
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: add10
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/add10"
  requirements:
  - triton-python


In [45]:
!seldon model load -f ./models/outlier-error.yaml 
!seldon model load -f ./models/add10.yaml 

{}
{}


In [46]:
!seldon model status --model-name outlier-error -w ModelAvailable | jq .
!seldon model status --model-name add10 -w ModelAvailable | jq .

{
  "modelName": "outlier-error",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:51:12.225612899Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:51:12.225612899Z"
      },
      "modelDefn": {
        "meta": {
          "name": "outlier-error",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/outlier",
          "requirements": [
            "triton-python"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "add10",
  "versions": [
    {
      "version": 1,
      

In [47]:
!cat ./pipelines/outlier.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: outlier
spec:
  steps:
    - name: outlier-error
    - name: add10
      triggers:
      - outlier-error
  output:
    steps:
    - add10


In [48]:
!seldon pipeline load -f ./pipelines/outlier.yaml

{}


In [49]:
!seldon pipeline status -p outlier -w PipelineReady | jq .

{
  "pipelineName": "outlier",
  "versions": [
    {
      "pipeline": {
        "name": "outlier",
        "steps": [
          {
            "name": "add10",
            "triggers": [
              "outlier-error.outputs"
            ]
          },
          {
            "name": "outlier-error"
          }
        ],
        "output": {
          "steps": [
            "add10.outputs"
          ]
        },
        "kubernetesMeta": {}
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-04-26T08:51:45.834103459Z"
      }
    }
  ]
}


In [52]:
!seldon pipeline infer -p outlier --inference-mode grpc \
    '{"model_name":"outlier","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT",
      "datatype": "FP32",
      "shape": [
        "4"
      ],
      "contents": {
        "fp32Contents": [
          11,
          12,
          13,
          14
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AAAwQQAAQEEAAFBBAABgQQ=="
  ]
}


In [53]:
!seldon pipeline infer -p outlier --inference-mode grpc \
    '{"model_name":"outlier","inputs":[{"name":"INPUT","contents":{"fp32_contents":[100,2,3,4]},"datatype":"FP32","shape":[4]}]}' | jq .

Error: rpc error: code = FailedPrecondition desc = rpc error: code = Internal desc = Failed to process the request(s) for model instance 'outlier-error_1_0', message: TritonModelException: Outlier. Input sums to greater than 100

At:
  /mnt/agent/models/outlier-error_1/1/model.py(108): execute

Usage:
  seldon pipeline infer [flags]

Flags:
  -f, --file-path string        inference payload file
  -h, --help                    help for infer
      --inference-host string   seldon inference host (default "0.0.0.0")
      --inference-mode string   inference mode rest or grpc (default "rest")
      --inference-port int      seldon scheduler port (default 9000)
  -i, --iterations int          inference iterations (default 1)
  -p, --pipeline-name string    pipeline name for inference

Global Flags:
  -r, --show-request    show request
  -o, --show-response   show response (default true)

parse error: Invalid numeric literal at line 1, column 4


In [54]:
!seldon pipeline unload -p outlier

{}


In [55]:
!seldon model unload --model-name outlier-error
!seldon model unload --model-name add10

{}
{}


## Model Join with Trigger

In [56]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml
!cat ./models/check.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple3
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: check
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/check"
  requirements:
  - triton-python


In [57]:
!seldon model load -f ./models/tfsimple1.yaml 
!seldon model load -f ./models/tfsimple2.yaml 
!seldon model load -f ./models/tfsimple3.yaml 
!seldon model load -f ./models/check.yaml 

{}
{}
{}
{}


In [58]:
!seldon model status --model-name tfsimple1 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple2 -w ModelAvailable | jq .
!seldon model status --model-name tfsimple3 -w ModelAvailable | jq .
!seldon model status --model-name check -w ModelAvailable | jq .

{
  "modelName": "tfsimple1",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:52:03.954426584Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:52:03.954426584Z"
      },
      "modelDefn": {
        "meta": {
          "name": "tfsimple1",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/simple",
          "requirements": [
            "tensorflow"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "tfsimple2",
  "versions": [
    {
      "version": 1,
      "serverN

In [59]:
!cat ./pipelines/tfsimples-join-outlier.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: joincheck
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
    - name: check
      inputs:
      - tfsimple1.outputs.OUTPUT0
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT
    - name: tfsimple3      
      inputs:
      - tfsimple1.outputs.OUTPUT0
      - tfsimple2.outputs.OUTPUT1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple2.outputs.OUTPUT1: INPUT1
      triggers:
      - check.outputs.OUTPUT
  output:
    steps:
    - tfsimple3


In [60]:
!seldon pipeline load -f ./pipelines/tfsimples-join-outlier.yaml

{}


In [61]:
!seldon pipeline status -p joincheck -w PipelineReady | jq .

{
  "pipelineName": "joincheck",
  "versions": [
    {
      "pipeline": {
        "name": "joincheck",
        "steps": [
          {
            "name": "check",
            "inputs": [
              "tfsimple1.outputs.OUTPUT0"
            ],
            "tensorMap": {
              "tfsimple1.outputs.OUTPUT0": "INPUT"
            }
          },
          {
            "name": "tfsimple1"
          },
          {
            "name": "tfsimple2"
          },
          {
            "name": "tfsimple3",
            "inputs": [
              "tfsimple1.outputs.OUTPUT0",
              "tfsimple2.outputs.OUTPUT1"
            ],
            "tensorMap": {
              "tfsimple1.outputs.OUTPUT0": "INPUT0",
              "tfsimple2.outputs.OUTPUT1": "INPUT1"
            },
            "triggers": [
              "check.outputs.OUTPUT"
            ]
          }
        ],
        "output": {
          "steps": [
            "tfsimple3.outputs"
          ]
        },
        "kubernetesMeta"

In [64]:
!seldon pipeline infer -p joincheck --inference-mode grpc \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]},"datatype":"INT32","shape":[1,16]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2,
          2
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAA==",
    "AgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAA

In [65]:
!seldon pipeline infer -p joincheck --inference-mode grpc \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq .

Error: rpc error: code = FailedPrecondition desc = rpc error: code = Internal desc = Failed to process the request(s) for model instance 'check_1_0', message: TritonModelException: Outlier. Input sums to greater than 100

At:
  /mnt/agent/models/check_1/1/model.py(107): execute

Usage:
  seldon pipeline infer [flags]

Flags:
  -f, --file-path string        inference payload file
  -h, --help                    help for infer
      --inference-host string   seldon inference host (default "0.0.0.0")
      --inference-mode string   inference mode rest or grpc (default "rest")
      --inference-port int      seldon scheduler port (default 9000)
  -i, --iterations int          inference iterations (default 1)
  -p, --pipeline-name string    pipeline name for inference

Global Flags:
  -r, --show-request    show request
  -o, --show-response   show response (default true)

parse error: Invalid numeric literal at line 1, column 4


In [66]:
!seldon pipeline unload -p joincheck

{}


In [67]:
!seldon model unload --model-name tfsimple1
!seldon model unload --model-name tfsimple2
!seldon model unload --model-name tfsimple3
!seldon model unload --model-name check

{}
{}
{}
{}


## Pipeline Input Tensors
Access to indivudal tensors in pipeline inputs

In [68]:
!cat ./models/mul10.yaml
!cat ./models/add10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: mul10
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/mul10"
  requirements:
  - triton-python
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: add10
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/add10"
  requirements:
  - triton-python


In [69]:
!seldon model load -f ./models/mul10.yaml 
!seldon model load -f ./models/add10.yaml 

{}
{}


In [70]:
!seldon model status --model-name mul10 -w ModelAvailable | jq .
!seldon model status --model-name add10 -w ModelAvailable | jq .

{
  "modelName": "mul10",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-26T08:53:01.369704527Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-26T08:53:01.369704527Z"
      },
      "modelDefn": {
        "meta": {
          "name": "mul10",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/triton/mul10",
          "requirements": [
            "triton-python"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "add10",
  "versions": [
    {
      "version": 1,
      "serverName": "tri

In [71]:
!cat ./pipelines/pipeline-inputs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: pipeline-inputs
  namespace: seldon-mesh
spec:
  steps:
  - name: mul10
    inputs:
    - pipeline-inputs.inputs.INPUT0
    tensorMap:
      pipeline-inputs.inputs.INPUT0: INPUT
  - name: add10
    inputs:
    - pipeline-inputs.inputs.INPUT1
    tensorMap:
      pipeline-inputs.inputs.INPUT1: INPUT
  output:
    steps:
    - mul10
    - add10



In [72]:
!seldon pipeline load -f ./pipelines/pipeline-inputs.yaml

{}


In [73]:
!seldon pipeline status -p pipeline-inputs -w PipelineReady | jq .

{
  "pipelineName": "pipeline-inputs",
  "versions": [
    {
      "pipeline": {
        "name": "pipeline-inputs",
        "steps": [
          {
            "name": "add10",
            "inputs": [
              "pipeline-inputs.inputs.INPUT1"
            ],
            "tensorMap": {
              "pipeline-inputs.inputs.INPUT1": "INPUT"
            }
          },
          {
            "name": "mul10",
            "inputs": [
              "pipeline-inputs.inputs.INPUT0"
            ],
            "tensorMap": {
              "pipeline-inputs.inputs.INPUT0": "INPUT"
            }
          }
        ],
        "output": {
          "steps": [
            "mul10.outputs",
            "add10.outputs"
          ]
        },
        "kubernetesMeta": {
          "namespace": "seldon-mesh"
        }
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-04-26T08:53:26.92110949

In [75]:
!seldon pipeline infer -p pipeline-inputs --inference-mode grpc \
    '{"model_name":"pipeline","inputs":[{"name":"INPUT0","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]},{"name":"INPUT1","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' | jq .

{
  "outputs": [
    {
      "name": "OUTPUT",
      "datatype": "FP32",
      "shape": [
        "4"
      ],
      "contents": {
        "fp32Contents": [
          10,
          20,
          30,
          40
        ]
      }
    },
    {
      "name": "OUTPUT",
      "datatype": "FP32",
      "shape": [
        "4"
      ],
      "contents": {
        "fp32Contents": [
          11,
          12,
          13,
          14
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AAAgQQAAoEEAAPBBAAAgQg==",
    "AAAwQQAAQEEAAFBBAABgQQ=="
  ]
}


In [76]:
!seldon pipeline unload -p pipeline-inputs

{}


In [77]:
!seldon model unload --model-name mul10
!seldon model unload --model-name add10

{}
{}
